In [22]:
import nupack as nu
import math 

In [23]:
def sciprint(string:str):
    print('{:3e}'.format(string))

def kelvin(temperature):
    return temperature + 273.15

def gibbs_free(enthalpy, entropy, temperature): #[DG]=Kcal/mol
    entropy_kcal = entropy/1000 #[S]=cal/mol*K
    return (enthalpy - (temperature)*entropy_kcal) # [H]=Kcal/mol 
    
def k_equilibrium(DG, temperature):
    return math.exp(-(DG/(1.987e-3*(temperature))))   # (Kcal/mol)/((Kcal/mol*K)*K) -> adimensional

In [24]:
import sys 
sys.path.insert(0,'/home/marktas/TESI-CODE/hDNA')

In [25]:
from classes.complex import Complex
from classes.strand import Strand
from classes.model import Model
from classes.chamber import Chamber

In [26]:
numodel = nu.Model(celsius=26)

A = nu.Strand('ACACACACCA', name = 'trial')
B = nu.Strand('TGTGTGTGGT'[::-1], name = 'comp')
C = nu.Complex([A,B],name='c')

In [27]:
model = Model('dna', '3D')
a = Strand(model, 'ACACACACCA')
b = Strand(model, 'TGTGTGTGGT')

coils = Complex(model, a, b, '.'*10+'+'+'.'*10)
chamber = Chamber(model, a, b)

In [28]:
for sl in chamber.offcores:
    print(sl.consecutive_nucleations)
    print(sl.sequences)
    print(sl.structure)
    print('\n')

4
ACACACACCA+TGGTGTGTGT
((((......+..))))....


6
ACACACACCA+TGGTGTGTGT
((((((....+..))))))..


10
ACACACACCA+TGGTGTGTGT
((((((((((+))))))))))


6
ACACACACCA+TGGTGTGTGT
..((((((..+....))))))


4
ACACACACCA+TGGTGTGTGT
....((((..+......))))




In [29]:
nucleated1 = nu.Structure('((((......+..))))....') #L1
nucleated2 = nu.Structure('((((((....+..))))))..') #L2
nucleated3 = nu.Structure('..((((((..+....))))))') #R2
nucleated4 = nu.Structure('....((((..+......))))') #R1

dupxstruct = nu.Structure('((((((((((+))))))))))') #D

In [30]:
dG1 = nu.structure_energy(strands=[A,B], structure=nucleated1, model=numodel)
dG2 = nu.structure_energy(strands=[A,B], structure=nucleated2, model=numodel)
dG3 = nu.structure_energy(strands=[A,B], structure=nucleated3, model=numodel)
dG4 = nu.structure_energy(strands=[A,B], structure=nucleated4, model=numodel)

dGD = nu.structure_energy(strands=[A,B], structure=dupxstruct, model=numodel)


In [31]:
T = kelvin(37)

energies = (0, dG1, dG2, dG3, dG4, dGD)
#           S  L1   L2   R2   R1    D 
#           0   1    2    3    4    5

In [32]:
kfcollision = 2e7
kfsliding = 1e7 

In [33]:
# SS <--> L1
DG = energies[0] - energies[1]
KE = k_equilibrium(-DG,T)
Kf = kfcollision
Kb = Kf / KE
print(DG)
sciprint(KE)
print(Kb)

8.430052896237473
8.725702e+05
22.920792678077465


In [34]:
# SS <--> L2
DG = energies[0] - energies[2]
KE = k_equilibrium(-DG,T)
Kf = kfcollision
Kb = Kf / KE
print(DG)
sciprint(KE)
print(Kb)

13.350720315228285
2.561401e+09
0.007808225837362128


In [35]:
# SS <--> R2
DG = energies[0] - energies[3]
KE = k_equilibrium(-DG,T)
Kf = kfcollision
Kb = Kf / KE
print(DG)
sciprint(KE)
print(Kb)

12.067131390865127
3.190883e+08
0.0626785751957368


In [36]:
# SS <--> R1
DG = energies[0] - energies[4]
KE = k_equilibrium(-DG,T)
Kf = kfcollision
Kb = Kf / KE
print(DG)
sciprint(KE)
print(Kb)

7.146463971874315
1.087010e+05
183.99091641848813


In [37]:
# SS <--> D
DG = energies[0] - energies[5]
KE = k_equilibrium(-DG,T)
Kf = kfcollision
Kb = Kf / KE
print(DG)
sciprint(KE)
print(Kb)

20.52172103400262
2.897362e+14
6.902831840829526e-08


In [38]:
#L1 <--> L2
DG = energies[1] - energies[2]
KE = k_equilibrium(-DG,T)
Kf = kfsliding
Kb = Kf / KE
print(DG)
sciprint(KE)
print(Kb)

4.920667418990812
2.935467e+03
3406.6124793451313


In [39]:
# L2 <--> D
DG = energies[2] - energies[5]
KE = k_equilibrium(-DG,T)
Kf = kfsliding
Kb = Kf / KE
print(DG)
sciprint(KE)
print(Kb)

7.171000718774335
1.131163e+05
88.40461309148736


In [40]:
# R1 <--> R2
DG = energies[4] - energies[3]
KE = k_equilibrium(-DG,T)
Kf = kfsliding
Kb = Kf / KE
print(DG)
sciprint(KE)
print(Kb)

4.920667418990812
2.935467e+03
3406.6124793451313


In [41]:
# R2 <--> D
DG = energies[3] - energies[5]
KE = k_equilibrium(-DG,T)
Kf = kfsliding
Kb = Kf / KE
print(DG)
sciprint(KE)
print(Kb)

8.454589643137494
9.080125e+05
11.01306438328072


In [42]:
""" 

SS <--> L1 
energies(0) <--> energies(1)

SS <--> L2 
energies(0) <--> energies(2)

SS <--> R2 
energies(0) <--> energies(3)

SS <--> R1 
energies(0) <--> energies(4)

SS <--> D 
energies(0) <--> energies(5)

L1 <--> L2 
energies(1) <--> energies(2)

L2 <--> D
energies(2) <--> energies(5)

R1 <--> R2 
energies(4) <--> energies(3)

R2 <--> D
energies(3) <--> energies(5)
"""

' \n\nSS <--> L1 \nenergies(0) <--> energies(1)\n\nSS <--> L2 \nenergies(0) <--> energies(2)\n\nSS <--> R2 \nenergies(0) <--> energies(3)\n\nSS <--> R1 \nenergies(0) <--> energies(4)\n\nSS <--> D \nenergies(0) <--> energies(5)\n\nL1 <--> L2 \nenergies(1) <--> energies(2)\n\nL2 <--> D\nenergies(2) <--> energies(5)\n\nR1 <--> R2 \nenergies(4) <--> energies(3)\n\nR2 <--> D\nenergies(3) <--> energies(5)\n'